1-Sep-2019 to 12-Jul-2020

In [1]:
#Import relevant modules
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import re
import pandas as pd

In [2]:
#Initialize variable
#In this program, only data of 2019/2020 season is scraped
#We start scrapping with the race on 1 Sep 2019 and end at 12 Jul 2020
URL_racing_res = 'https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx?RaceDate=2019/09/01&RaceNo=1'
URL_date_suffix= re.search(r'\d{4}/\d{2}/\d{2}',URL_racing_res).group(0)
URL_race_no_suffix = re.search(r'\d+$',URL_racing_res).group(0)

In [3]:
#Please change the path to the location of your chromedriver
driver = webdriver.Chrome('/Users/Hei/Applications/chromedriver')


In [4]:
#Scrape data using chromedriver
#If the code fail to fetch enough html, please extend the sleep time
driver.get(URL_racing_res)
time.sleep(5)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, 'html.parser')

In [5]:
def create_pref_df():
    '''Create DataFrame from performance table data'''
    
    global soup
    soup.find_all(class_="f_fs12")
    
    #Get #Get all <td> elements in html
    list_performance = get_html_from_soup(soup)

    #Number of columns of performance data table
    num_of_columns=12
    #Number of cells in performance data table
    num_of_table_element=len(list_performance)
    #Number of horse
    num_of_horse = int(num_of_table_element/num_of_columns)
    
    #Create dictionary for storing perfotmance table data
    dict_pref = {'place':[],'horse_no':[],'horse':[],'jockey':[],'trainer':[],'actual_weight':[],\
                 'declared_horse_weight':[],'draw':[],'lbw':[],'running_position':[],'finish_time':[],\
                 'win_odds':[]}


    #Load performance table data into dataframe
    count=0
    for i in list_performance:
        if count%12==0:
            dict_pref['place'].append(i.text)
        elif count%12==1:
            dict_pref['horse_no'].append(i.text)
        elif count%12==2:
            dict_pref['horse'].append(i.text)
        elif count%12==3:
            dict_pref['jockey'].append(i.text)
        elif count%12==4:
            dict_pref['trainer'].append(i.text)
        elif count%12==5:
            dict_pref['actual_weight'].append(i.text)
        elif count%12==6:
            dict_pref['declared_horse_weight'].append(i.text)
        elif count%12==7:
            dict_pref['draw'].append(i.text)
        elif count%12==8:
            dict_pref['lbw'].append(i.text)
        elif count%12==9:
            dict_pref['running_position'].append(i.text)
        elif count%12==10:
            dict_pref['finish_time'].append(i.text)
        elif count%12==11:
            dict_pref['win_odds'].append(i.text)

        count+=1
    
    df_table=pd.DataFrame(dict_pref)
    
    return df_table,num_of_horse

def get_html_from_soup(soup):
    #Get all <td> elements in html
    data_performance=soup.find_all(class_="f_fs12")[1:-2][0]
    list_performance=data_performance.find_all('td')
    return list_performance

In [6]:
df_perf,num_of_horse = create_pref_df()
df_perf

,place,horse_no,horse,jockey,trainer,actual_weight,declared_horse_weight,draw,lbw,running_position,finish_time,win_odds
0,1,\n3,GOOD RUNNERS WAY(C127),\nZ Purton\n,\n\n ...,131,1072,6,-,\n\n\n3 ...,1:35.48,2.2
1,2,\n4,REGENCY GEM(C035),\nJ Moreira\n,\n\n ...,129,1179,7,SH,\n\n\n9 ...,1:35.50,4.9
2,3,\n7,PROUD SKY(V406),\nK Teetan\n,\n\n ...,121,1053,4,1/2,\n\n\n5 ...,1:35.57,18
3,4,\n2,LE PANACHE(V369),\nA Sanna\n,\n\n ...,132,1076,3,1-1/4,\n\n\n4 ...,1:35.68,5.7
4,5,\n1,GOLD VELVET(V400),\nL Hewitson\n,\n\n ...,133,1155,8,1-1/2,\n\n\n8 ...,1:35.71,7
5,6,\n9,ROMANTIC JOURNEY(B107),\nH T Mo\n,\n\n ...,112,1085,1,1-3/4,\n\n\n1 ...,1:35.77,19
6,7,\n5,WICKER(B425),\nN Callan\n,\n\n ...,126,1118,9,2,\n\n\n7 ...,1:35.79,50
7,8,\n6,WHAMPOA STAR(B146),\nM F Poon\n,\n\n ...,119,1139,5,7-1/2,\n\n\n2 ...,1:36.67,14
8,9,\n8,YOUNG GLORY(B292),\nM Chadwick\n,\n\n ...,120,1207,2,10-1/2,\n\n\n6 ...,1:37.15,21


In [7]:
def create_race_info_df():
    global soup,num_of_horse
    
    table = soup.find_all(class_="race_tab")
    table_data = [i.find_all('td')for i in table]
    l = "".join([i.text for i in table_data[0]])

    d=[[re.findall("RACE\s(\d+)", l)[0]+re.findall(r"RACE.+(\(\d+\))", l)[0],re.findall("Class\s(\d+)", l)[0]+
       re.findall(r"Class.+(\- \d+M \- \(\d+\-\d+\))", l)[0],
      re.findall("Going \:(HEAVY|SOFT|GOOD TO SOFT|GOOD|GOOD TO FIRM|FIRM)\w+", l)[0],
       re.findall("TURF \- (\"\w\" Course)", l)[0],
       re.findall("HK\$ \d+\,\d*\,*\d*",l)[0]] for i in range(num_of_horse)]

    df=pd.DataFrame(d)
    df.columns=('Race','Class','Going','Turf','Prize')
    
    return df

In [8]:
df_race_info=create_race_info_df()

In [9]:
df_race_info

,Race,Class,Going,Turf,Prize
0,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
1,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
2,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
3,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
4,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
5,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
6,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
7,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
8,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"


In [10]:
def create_data_df(performance,race_info):
    '''Create a DataFrame containing performance and race info data'''
    df_data=pd.concat([performance,race_info],axis=1)
    return df_data

In [11]:
df_data = create_data_df(df_perf,df_race_info)

In [12]:
df_data

,place,horse_no,horse,jockey,trainer,actual_weight,declared_horse_weight,draw,lbw,running_position,finish_time,win_odds,Race,Class,Going,Turf,Prize
0,1,\n3,GOOD RUNNERS WAY(C127),\nZ Purton\n,\n\n ...,131,1072,6,-,\n\n\n3 ...,1:35.48,2.2,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
1,2,\n4,REGENCY GEM(C035),\nJ Moreira\n,\n\n ...,129,1179,7,SH,\n\n\n9 ...,1:35.50,4.9,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
2,3,\n7,PROUD SKY(V406),\nK Teetan\n,\n\n ...,121,1053,4,1/2,\n\n\n5 ...,1:35.57,18,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
3,4,\n2,LE PANACHE(V369),\nA Sanna\n,\n\n ...,132,1076,3,1-1/4,\n\n\n4 ...,1:35.68,5.7,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
4,5,\n1,GOLD VELVET(V400),\nL Hewitson\n,\n\n ...,133,1155,8,1-1/2,\n\n\n8 ...,1:35.71,7,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
5,6,\n9,ROMANTIC JOURNEY(B107),\nH T Mo\n,\n\n ...,112,1085,1,1-3/4,\n\n\n1 ...,1:35.77,19,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
6,7,\n5,WICKER(B425),\nN Callan\n,\n\n ...,126,1118,9,2,\n\n\n7 ...,1:35.79,50,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
7,8,\n6,WHAMPOA STAR(B146),\nM F Poon\n,\n\n ...,119,1139,5,7-1/2,\n\n\n2 ...,1:36.67,14,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"
8,9,\n8,YOUNG GLORY(B292),\nM Chadwick\n,\n\n ...,120,1207,2,10-1/2,\n\n\n6 ...,1:37.15,21,1(1),5- 1600M - (40-0),GOOD,"""B"" Course","HK$ 725,000"


In [13]:
def get_num_of_race():
    '''Find out the number of image for each race'''
    pattern_img = re.compile('.+src="/racing/info/StaticFile/Images/Racing/racecard_rt.+')
    img_list = [ str(tag) for tag in soup.find_all('img') if pattern_img.match(str(tag)) ]
    return len(img_list)

In [16]:
def get_next_race():
    '''return the url of next race'''
    global soup,URL_date_suffix,URL_race_no_suffix
    
    #get next race no. suffix in url
    max_race = get_num_of_race()
    if int(URL_race_no_suffix)+1<=max_race:
        next_race_no_suffix = str(int(URL_race_no_suffix)+1)
        next_date_suffix = URL_date_suffix
        print(next_date_suffix,next_race_no_suffix)
    else:
        #get next date suffix in url
        date = soup.find(class_="f_fs11")
        date_selection = date.text.replace('\n',',')[1:-1].split(',')
        cur_race_day = URL_date_suffix[-2:]+URL_date_suffix[4:8]+URL_date_suffix[:4]
        next_race_day = date_selection[date_selection.index(cur_race_day)-1]
        next_date_suffix = next_race_day[-4:]+next_race_day[2:6]+next_race_day[:2]
        next_race_no_suffix = 1
        print(next_date_suffix,next_race_no_suffix)
    
    next_race_url='https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx?RaceDate='+next_date_suffix+'&RaceNo='+next_race_no_suffix
    return next_race_url

In [17]:
get_next_race()

2019/09/01 2


'https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx?RaceDate=2019/09/01&RaceNo=2'

In [ ]:
URL_date_suffix,URL_race_no_suffix

In [18]:
URL_racing_res

'https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx?RaceDate=2019/09/01&RaceNo=1'